# 1. Install Dependencies and Setup

In [ ]:
pip install tensorflow  opencv-python matplotlib

In [ ]:
pip install -U scikit-learn scipy matplotlib

In [ ]:
!pip install seaborn

In [ ]:
!pip list

In [ ]:
import tensorflow as tf
import os

# 2. Remove dodgy images

In [ ]:
import cv2
import imghdr

In [ ]:
data_dir = 'D:\ImageClassification\data' 

In [ ]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

# 3. Load Data

In [ ]:
import numpy as np 

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

data_dir = 'D:\ImageClassification\data'
class_labels = ['cloudy', 'dew', 'fogsmog', 'glaze', 'hail', 'lightining', 'rain', 'rainbow', 'rime','shine', 'snow', 'sunrise']

# Veri kümesini oluştur
data = tf.keras.utils.image_dataset_from_directory(data_dir)

# Veri kümesinin bir örnek batch'ini alma
batch = next(iter(data))

# Batch içindeki görüntüler ve etiketler
images, labels = batch

# Her bir sınıf için ilk görüntüyü gösterme
for class_index, class_label in enumerate(class_labels):
    class_indices = tf.where(labels == class_index).numpy().flatten()
    if len(class_indices) > 0:
        image_for_class = images[class_indices[0]]
        plt.imshow(image_for_class.numpy().astype(int))
        plt.title(f'Class: {class_label}')
        plt.show()


In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
#2 satır 6 sütün şeklinde rastgele 12 veriyi gösterme
fig, ax = plt.subplots(nrows=2, ncols=6, figsize=(20, 6))

# Veri kümesinden 12 resmi gösterme
for i in range(2):
    for j in range(6):
        img_index = i * 6 + j
        if img_index < 12:
            ax[i, j].imshow(batch[0][img_index].astype(int))
            ax[i, j].title.set_text(batch[1][img_index])

plt.show()


# 4. Scale Data

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
# Veri setini normalize etme ve one-hot encoding uygulama
data = data.map(lambda x, y: (x / 255, tf.one_hot(y, depth=12)))


# 5. Split Data

In [ ]:
train_size = int(len(data)*0.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [ ]:
print(train_size)
print(val_size)
print(test_size)

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
from collections import Counter
class_labels = ['cloudy', 'dew', 'fogsmog', 'glaze', 'hail', 'lightining', 'rain', 'rainbow', 'rime', 'shine', 'snow', 'sunrise']

# Train setindeki sınıfların örnek sayıları
train_class_counts = Counter([class_labels[np.argmax(label.numpy())] for _, label in train])

# Test setindeki sınıfların örnek sayıları
test_class_counts = Counter([class_labels[np.argmax(label.numpy())] for _, label in test])

print("Train seti sınıf örnek sayıları:")
print(train_class_counts)

print("\nTest seti sınıf örnek sayıları:")
print(test_class_counts)


# 6. Build Deep Learning Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,BatchNormalization

In [ ]:
num_classes = 12 # Toplam sınıf sayısı


In [ ]:
input_shape = (256, 256, 3)
num_classes = 12

model = Sequential()

# Convolutional Block 1
model.add(Conv2D(96, (11, 11), strides=(4, 4), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

# Convolutional Block 2
model.add(Conv2D(256, (5, 5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

# Convolutional Block 3
model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())

# Convolutional Block 4
model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())

# Convolutional Block 5
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

# Flatten and Dense Layers
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(num_classes, activation='softmax'))

# Print the model summary
model.summary()

In [ ]:
model.compile('adam', loss=tf.losses.CategoricalCrossentropy(), metrics=['accuracy'])

# 7. Train

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
hist = model.fit(train, epochs=50, validation_data=val,  callbacks=[tensorboard_callback, early_stopping])

In [ ]:
from keras.callbacks import TensorBoard
import time
kerasboard = TensorBoard(log_dir="/tmp/tensorboard/{}".format(time),
                        batch_size=32,
                        histogram_freq=1,
                        write_grads=True)

In [ ]:
print("tensorboard --logdir="+kerasboard.log_dir)

# 8. Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 9. Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy,Accuracy
from tensorflow.keras.utils import to_categorical

In [ ]:
model.load_weights('D:\ImageClassification\models\h.h5')

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

# Reset states to clear previous values
pre.reset_states()
re.reset_states()
acc.reset_states()

# Evaluate the model on the validation dataset
for inputs, targets in val:
    predictions = model(inputs)
    pre.update_state(targets, predictions)
    re.update_state(targets, predictions)
    acc.update_state(targets, predictions)

# Get the final values
precision_value = pre.result().numpy()
recall_value = re.result().numpy()
accuracy_value = acc.result().numpy()

# Print the values
print(f'Precision: {precision_value}')
print(f'Recall: {recall_value}')
print(f'Categorical Accuracy: {accuracy_value}')

In [ ]:
num_classes =12
precisions = [Precision() for _ in range(num_classes)]
recalls = [Recall() for _ in range(num_classes)]
accuracies = [CategoricalAccuracy() for _ in range(num_classes)]
f1_scores = []
# Reset states to clear previous values
for metric in precisions + recalls + accuracies:
    metric.reset_states()

# Evaluate the model on the validation dataset
for inputs, targets in val:
    predictions = model(inputs)

    # Update metrics for each class
    for i in range(num_classes):
        precisions[i].update_state(targets[:, i], predictions[:, i])
        recalls[i].update_state(targets[:, i], predictions[:, i])
        accuracies[i].update_state(targets[:, i], predictions[:, i])

# Print the final values for each class
for i in range(num_classes):
    precision_value = precisions[i].result().numpy()
    recall_value = recalls[i].result().numpy()
    accuracy_value = accuracies[i].result().numpy()
    
    f1_score_value = 2 * (precision_value * recall_value) / (precision_value + recall_value + 1e-8)
    f1_scores.append(f1_score_value)


    print(f'Class {i + 1}: Precision: {precision_value}, Recall: {recall_value}, Accuracy: {accuracy_value}, F1 Score: {f1_score_value}')
    
# Calculate and print the overall F1 Score
overall_f1_score = sum(f1_scores) / num_classes
print(f'Overall F1 Score: {overall_f1_score}')

In [ ]:
test = data.skip(train_size + val_size).take(test_size)

In [ ]:
accuracy_metric = Accuracy()

for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    acc.update_state(y, yhat)
    accuracy_metric.update_state(tf.argmax(y, axis=1), tf.argmax(yhat, axis=1))


test_accuracy = accuracy_metric.result().numpy()
print(f'Test Accuracy: {test_accuracy}')

# 10. Test

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Görüntüyü okuma ve uygun formatta yükleme
img = cv2.imread('denemeVeri/my5.jpg')

# Görüntü boş mu kontrolü
if img is not None:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # OpenCV, görüntüyü BGR formatında okur, bu nedenle RGB'ye dönüştürüyoruz

    # Görüntüyü gösterme
    plt.imshow(img)
    plt.show()
else:
    print("Görüntü yüklenemedi.")
    

if img is not None:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resize = tf.image.resize(img, (256, 256))
    resize = np.expand_dims(resize / 255, 0)  

else:
    print(f"Error: Unable to read the image at {img_path}")
    
class_names = ['cloudy', 'dew', 'fogsmog', 'glaze', 'hail', 'lightning', 'rain', 'rainbow', 'rime', 'shine', 'snow', 'sunrise']
predictions = model.predict(resize)
predicted_class = np.argmax(predictions, axis=-1)

# Print some debug information
print(f'Predicted class indices: {predicted_class}')
if 0 <= predicted_class[0] < len(class_names):
    predicted_class_label = class_names[predicted_class[0]]
    print(f'Predicted class: {predicted_class_label}')
else:
    print('Error: Predicted class index is out of range.')

# Print class probabilities
for i, prob in enumerate(predictions[0]):
    class_label = class_names[i]
    print(f'Probability for class {class_label}: {prob}')



In [ ]:
# Görüntüyü okuma ve uygun formatta yükleme
img = cv2.imread('denemeVeri/benimCektigim2.jpg')

# Görüntü boş mu kontrolü
if img is not None:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # OpenCV, görüntüyü BGR formatında okur, bu nedenle RGB'ye dönüştürüyoruz

    # Görüntüyü gösterme
    plt.imshow(img)
    plt.show()
else:
    print("Görüntü yüklenemedi.")

In [ ]:
img_path = 'denemeVeri/my8.jpg'
img = cv2.imread(img_path)

if img is not None:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resize = tf.image.resize(img, (256, 256))
    resize = np.expand_dims(resize / 255, 0)  

else:
    print(f"Error: Unable to read the image at {img_path}")


In [ ]:
class_names = ['cloudy', 'dew', 'fogsmog', 'glaze', 'hail', 'lightning', 'rain', 'rainbow', 'rime', 'shine', 'snow', 'sunrise']
predictions = model.predict(resize)
predicted_class = np.argmax(predictions, axis=-1)

# Print some debug information
print(f'Predicted class indices: {predicted_class}')
if 0 <= predicted_class[0] < len(class_names):
    predicted_class_label = class_names[predicted_class[0]]
    print(f'Predicted class: {predicted_class_label}')
else:
    print('Error: Predicted class index is out of range.')

# Print class probabilities
for i, prob in enumerate(predictions[0]):
    class_label = class_names[i]
    print(f'Probability for class {class_label}: {prob}')


# 11. Save the Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','h.h5'))

In [ ]:
new_model = load_model('D:\ImageClassification\models\h.h5')

# 12. Confusion Matrix

In [ ]:
# Confusion Matrix oluşturma ve çizme
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd

# Test veri seti üzerinde tahminlerin ve gerçek etiketlerin toplanması
y_true = []
y_pred = []

for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    
    # Sınıf indekslerini listeler içinde ekleyin
    y_true.append([np.argmax(label) for label in y])
    y_pred.append([np.argmax(pred) for pred in yhat])

# Listeleri numpy dizilerine çevirme
y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)

# Confusion Matrix oluşturma
conf_mat = confusion_matrix(y_true, y_pred)

# Sınıf isimleri
class_names = ['cloudy', 'dew', 'fogsmog', 'glaze', 'hail', 'lightning', 'rain', 'rainbow', 'rime', 'shine', 'snow', 'sunrise']

# Pandas DataFrame'e dönüştürme
conf_mat_df = pd.DataFrame(conf_mat, index=class_names, columns=class_names)

# Confusion Matrix'i çizme
plt.figure(figsize=(12, 10))
sns.heatmap(conf_mat_df, annot=True, fmt='g', cmap='Blues', cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()





# Realtime

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input
import time

# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8
sequence_length = 1

# [10,1,224,224,3]-needed input shape
cap = cv2.VideoCapture(0)

# Süreyi beklemek için kullanılacak zaman aralığı (saniye cinsinden)
wait_interval = 0.1

# Önceki tahminin başlangıçta tanımlanması
previous_prediction = None

while cap.isOpened():
    # Read feed
    ret, frame = cap.read()

    # Resize the frame to match the model's expected sizing
    resized_frame = cv2.resize(frame, (256, 256))

    # Preprocess the frame for the model
    input_data = preprocess_input(np.expand_dims(resized_frame, axis=0))

    # Update the sequence with the preprocessed frame
    sequence.append(input_data)

    # Eğer belirlenen saniye aralığına ulaşıldıysa işlem yap
    if len(sequence) == sequence_length:
        sequence = np.array(sequence)

        # Remove the singleton dimension
        sequence = np.squeeze(sequence, axis=1)

        # Doğrudan modeli çağırın ve tahmin alın
        res = model.predict(sequence)
        current_prediction = class_names[np.argmax(res)]

        # Eğer önceki tahminden farklıysa güncelle
        if current_prediction != previous_prediction:
            print(current_prediction)

        # Güncelleme yapıldıktan sonra ekrandaki etiketi güncelle
        sentence = [current_prediction]

        # Görsel olarak etiketi belirtmek için metni ayarla
        text = ' '.join(sentence)
        font = cv2.FONT_HERSHEY_SIMPLEX
        org = (10, 30)
        font_scale = 1
        coloqr = (0, 0, 255)  # Kırmızı renk (BGR formatında)
        thickness = 2
        frame = cv2.putText(frame, text, org, font, font_scale, color, thickness, cv2.LINE_AA)

        # Show the frame
        cv2.imshow('OpenCV Feed', frame)

        # Bekleme süresi kadar uyku
        time.sleep(wait_interval)

        # Reset the sequence for the next predictions
        sequence = []

        # Güncelleme yapıldıktan sonra önceki tahmini güncelle
        previous_prediction = current_prediction

    # Break gracefully
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
